# First-Order Sigma-Delta Modulator (Behavioral)

Switched-capacitor first-order delta-sigma modulator using the inverter-based
auto-zeroing integrator topology (Chae & Han, JSSC 2009).

Behavioral model with ideal OTA, switches, and comparator.
Sine input, transient simulation, PSD and SNR/SNDR evaluation.

In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import Span

from src.runners.sigma_delta_runner import SigmaDeltaRunner

output_notebook()

Loading BokehJS ...

## Parameters

Set sampling frequency and OSR first (these define the signal bandwidth).
Input frequency must be well below the bandwidth to see harmonics.

In [2]:
VDD = 1.2
FS = 128e3            # Sampling clock (Hz)
OSR = 64              # Oversampling ratio
N_PERIODS = 8192/4      # Clock cycles to simulate
FFT_BIN_IDX = 21       # Desired FFT bin for input signal
FIN = FFT_BIN_IDX * FS / N_PERIODS  # Input sine frequency, << BW
AMPL = VDD / 20            # Sine amplitude (peak) around VDD/2
N_BUFF_PERIODS = 40   # Number of buffer periods
runner = SigmaDeltaRunner(vdd=VDD, fin=FIN, ampl=AMPL, fs=FS, osr=OSR)
print(f'Bandwidth: {runner.bw:.0f} Hz')
print(f'FIN/BW:    {FIN / runner.bw:.2f}')

Bandwidth: 1000 Hz
FIN/BW:    1.31


## Run Simulation

In [3]:
# result = runner.run_and_evaluate(n_periods=10)
result = runner.run_and_evaluate(n_periods=N_PERIODS + N_BUFF_PERIODS)

[INFO] Netlist written to /workspaces/pade/examples/tutorials/../sim_data/sigma_delta/run/SigmaDeltaTranTB.spice
[INFO] Running: ngspice -b /workspaces/pade/examples/tutorials/../sim_data/sigma_delta/run/SigmaDeltaTranTB.spice -r /workspaces/pade/examples/tutorials/../sim_data/sigma_delta/run/output.raw


[ERROR] NGspice failed after 67.1s
[ERROR] See /workspaces/pade/examples/tutorials/../sim_data/sigma_delta/run/ngspice.out for details


RuntimeError: NGspice simulation failed

## Time-Domain Waveforms


In [ ]:
time = result['time']
T = 1 / FS
t0 = 0 * T  # skip startup
t1 = t0 + 300 * T
mask = (time >= t0) & (time <= t1)
t_us = time[mask] * 1e6

p1 = figure(title='Input (Vin)', x_axis_label='Time (\u00b5s)', y_axis_label='V',
            width=800, height=200)
p1.line(t_us, result['vin'][mask], line_width=1.5, color='steelblue')

p2 = figure(title='Integrator Output (Vint)', x_axis_label='Time (\u00b5s)', y_axis_label='V',
            width=800, height=250, x_range=p1.x_range)
p2.line(t_us, result['vint'][mask], line_width=1.5, color='firebrick')

p3 = figure(title='Comparator Output (Out)', x_axis_label='Time (\u00b5s)', y_axis_label='V',
            width=800, height=250, x_range=p1.x_range)
p3.line(t_us, result['out'][mask], line_width=1.5, color='seagreen')

p4 = figure(title='Clock Phases (Phi1, Phi2)', x_axis_label='Time (\u00b5s)', y_axis_label='V',
            width=800, height=200, x_range=p1.x_range)
p4.line(t_us, result['phi1'][mask], line_width=1.5, color='purple', legend_label='Phi1')
p4.line(t_us, result['phi2'][mask], line_width=1.5, color='orange', legend_label='Phi2')
p4.legend.location = 'top_right'

show(column(p1, p2, p3, p4))

## Output PSD and Metrics

In [ ]:
# freq, psd_dB, metrics = runner.psd_and_metrics(result)

# p = figure(title='Output PSD', x_axis_label='Frequency (Hz)',
#            y_axis_label='PSD (dB)', width=800, height=400)
# p.line(freq, psd_dB, line_width=1, color='steelblue', legend_label='PSD')

# # Mark signal bandwidth
# band_line = Span(location=runner.bw, dimension='height', line_color='red',
#                  line_dash='dashed', line_width=1)
# p.add_layout(band_line)
# p.legend.location = 'top_right'
# p.x_range.start = 0
# p.x_range.end = FS / 2

# show(p)

# print(f'Bandwidth: 0 - {runner.bw:.0f} Hz')
# print(f"SNR   = {metrics['SNR_dB']:.1f} dB")
# print(f"SNDR  = {metrics['SNDR_dB']:.1f} dB")
# print(f"ENOB  = {metrics['ENOB']:.1f} bits")
# print(f"HD2   = {metrics['HD2_dBc']:.1f} dBc")
# print(f"HD3   = {metrics['HD3_dBc']:.1f} dBc")
# print(f"FFT points: {metrics['n_fft']}")